In [1]:
import os

In [2]:
%pwd

'c:\\Users\\hp\\OneDrive\\Desktop\\e2e\\end-to-end-ml\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\hp\\OneDrive\\Desktop\\e2e\\end-to-end-ml'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



In [14]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from mlproject import logger
from mlproject.utils.common import get_size

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        """
        Zip_file_path: str
        Extracts the downloaded zip file into to the data directory
        Function returns home
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-03 10:53:11,262: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-03 10:53:11,278: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-03 10:53:11,285: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-03 10:53:11,289: INFO: common: created directory at: artifacts]
[2025-03-03 10:53:11,291: INFO: common: created directory at: artifacts/data_ingestion]


[2025-03-03 10:53:13,353: INFO: 3515850659: artifacts/data_ingestion/data.zip download with following info: 
Connection: close
Content-Length: 21984
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "85a38be3d4c70e3b6dc97fc47873623da88c9b850e1526aeb57ff236bdd2815c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 095C:2D4439:7FA700:12F4ACC:67C58A19
Accept-Ranges: bytes
Date: Mon, 03 Mar 2025 10:53:13 GMT
Via: 1.1 varnish
X-Served-By: cache-lcy-eglc8600043-LCY
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1740999193.127548,VS0,VE168
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: a20df5ea08d5efa4c24775aaf6f79cc4a6fe0a30
Expires: Mon, 03 Mar 2025 10:58:13 GMT
Source-Age: 0

]
